# 1.3 신경망의 학습
신경망으로 좋은 추론을 하기 위해서는 학습을 먼저 수행하고, 학습된 매개변수를 이용해 추론을 수행하는 흐름이 일반적이다.

한편 신경망의 학습은 최적의 매개변수값을 찾는 작업이다.

이번절에서는 신경망의 학습에 대해 살펴본다.

## 1.3.1 손실 함수

신경망 학습에는 학습이 얼마나 잘 이루어지고 있는지를 나타내기 위한 '척도'가 필요하다.<br>
일반적으로 학습 단계의 특정 시점에서 신경망의 성능을 나타내는 척도로 **손실(Loss)**를 사용한다.<br>
학습시 주어진 정답 데이터와 신경망이 예측한 결과를 비교해 예측이 얼마나 나쁜가를 산출한 단일 스칼라값(loss)를 구할 수 있다.


* 학습의 척도로 Loss를 사용한다.
* Loss는 정답 데이터와 신경망 예측 결과를 비교한 단일 스칼라값으로 구할 수 있다.

신경망의 손실은 **손실 함수(loss function)**을 사용해서 구한다.

다중 클래스 분류(multi-class classification) 신경망에서는 손실함수로 흔히 **교차 엔트로피 오차(Cross Entropy Error)**를 이용한다.

**손실함수를 적용한 신경망 계층 구성**

![alt text](./img/TwoLayerNetwork-with-LossFunction.PNG "TwoLayerNet-with-LossFunction")

-X: 입력데이터<br>
-t: 정답레이블<br>
-L: 손실<br>
-Softmax계층의 출력: 확률<br>
-Cross Entropy Error 계층: Softmax 계층 출력 확률과 정답 레이블이 입력

**Softmax 함수**<br>

![alt text](./img/Softmax-function.PNG)

* Softmax함수 출력의 각 원소는 0.0이상 1.0이하의 실수이다. <br>
* 원소들을 모두 더하면 1.0이 된다.
* 이 때문에 Softmax의 출력을 '확률'로 해석할 수 있다.

In [1]:
import numpy as np

a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)
print(exp_a)

sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

[ 1.34985881 18.17414537 54.59815003]
74.1221542101633
[0.01821127 0.24519181 0.73659691]


* Softmax 함수 구현

In [2]:
# Softmax 함수 정의

def softmax(s):
    c = np.max(s)
    exp_s = np.exp(s-c)
    sum_exp_s = np.sum(exp_s)
    y = exp_s / sum_exp_s
    return y

In [3]:
s = np.array([0.3, 2.9, 4.0])

print(softmax(s))

[0.01821127 0.24519181 0.73659691]


* Softmax 함수의 특징
    * 함수의 출력은 0에서 1.0사이이다.
    * 함수 출력의 총 합은 1이다.
    * Softmax함수를 적용해도 각 원소의 대소 관계는 변하지 않는다.

**CEE(Cross Entropy Error)**<br>
![alt text](./img/Cross-Entropy.PNG "Cross-Entropy function")
* tk는 k번째 클래스에 해당하는 정답 레이블이다.<br>
* log는 네이피어 상수(혹은 오일러의수)e를 밑으로 하는 로그이다.<br>
* 정답 레이블은 t=[0, 0, 1]과 같이 원-핫 벡터로 표현한다.

**Minibatch를 적용한 CrossEntropyError**<br>
![alt text](./img/Minibatch-CrossEntropy.PNG)

* tnk는 n번째 데이터에서 k차원째의 값을 의미한다.<br>
* ynk는 신경망의 출력이고, tnk는 정답 레이블이다.

**Softmax-with-Loss Layer**<br>
![alt text](./img/Softmax-with-Loss-Layer.png)

본 책에서는 소프트맥스 함수와 교차 엔트로피 오차를 계산하는 계층을 Softmax with Loss 계층 하나로 구현한다.

## 1.3.2 미분과 기울기

신경망 학습의 목표는 손실을 최소화하는 매개변수를 찾는 것이다.

이 때 중요한 것이 바로 **'미분'**과 **'기울기'**이다.

이번 절에서는 미분과 기울기에 대해 간략히 설명한다.

* Gradient(기울기)란 무엇인가
-x에 관한 y의 미분은 dy/dx라고 쓴다<br>
-이것이 의미하는 것은 x를 조금 변화시켰을 때(조금의 변화를 극한까지 줄일 때) y값이 얼마나 변하는가이다.

* 두 가지의 Gradient
    * 벡터의 각 원소에 대한 미분을 정리
    * 행렬에서의 기울기

* 벡터의 기울기

L은 스칼라, x는 벡터인 함수 L = f(x) 가 있다.

이 때 xi(x의 i번 째 원소)에 대한 미분과 x의 다른 원소의 미분을 다음과 같이 정리할 수 있다.

![alt text](./img/gradient-vector.PNG)

이처럼 벡터의 각 원소에 대한 미분을 정리한 것이 기울기이다.

* 행렬의 기울기

![alt text](./img/gradient-matrix.PNG)

* 행렬의 기울기는 원래의 행렬과 형상이 같다.
  
이 성질을 이용하면 매개변수 갱신과 연쇄 법칙을 쉽게 구현할 수 있다.

### 1.3.3 연쇄 법칙

학습 시 신경망은 학습데이터를 통해 손실(Loss)을 출력한다. 이 때 우리가 알고 싶은 것은 바로 **매개변수에 대한 손실의 기울기**이다.

Gradient of parameter?

* 연쇄 법칙(Chain-Rule)

오차역전파법을 이해하는 열쇠는 **연쇄법칙(Chain-Rule)**이다. 연쇄법칙이란 합성함수에 대한 미분의 법칙이다.

* 연쇄법칙이 중요한 이유

아무리 복잡하고, 많은 함수를 사용하더라도 개별 미분들을 이용해 전체 미분값을 구할 수 있기 때문이다.<br>
즉, 각 함수의 국소적 미분을 구할 수 있다면 그 값들을 곱해서 전체 미분을 구할 수 있다.

* Chain-Rule을 활용한 역전파 미분값 구하기

Chain-Rule에 따르면 역전파로 흐르는 미분값은 상류로부터 흘러온 미분과 각 연산 노드의 국소적인 미분을 곱해 계산할 수 있다.

## 1.3.4 계산 그래프

### 분기노드

![alt text](./img/분기노드.PNG)

분기노드는 위의 그림과 같이 분기하는 노드이다.

분기노드의 역전파는 (상류노드에서 흘러들어온) 분기노드의 합이다.

### Repeat Node

2개의 분기 노드를 일반화 하면 N개의 분기가 된다. 이를 Repeat Node라고 한다.

![alt text](./img/Repeat-Node.PNG)

분기노드 코드 구현

In [4]:
import numpy as np

D, N = 8, 7
x = np.random.randn(1, D)  # 입력
# y = np.random.randn(8, 7)
y = np.repeat(x, N, axis=0)  # 순전파

# 역전파
dy = np.random.randn(N,D)
dx = np.sum(dy, axis=0, keepdims=True)  # Keppdims= Ture
                                        # If this is set to True, the axes which are reduced are left
                                        # in the result as dimensions with size one. With this option,
                                        # the result will broadcast correctly against the input array.

### Sum Node

## 1.3.5 기울기 도출과 역전파 구현

## 1.3.6 가중치 갱신

## 정리